In [1]:
import sys 
import os 
sys.path.insert(0, os.path.dirname(os.getcwd()))

In [2]:
from utils import get_connection
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
con = get_connection(database='xiaohongshu')

connecting to rm-uf6lfxk83h7onjmwvjo.mysql.rds.aliyuncs.com3306/xiaohongshu


In [4]:
# pd.read_sql("show tables ",con)

In [5]:
dict_sql = """
select tag_category, tag_value, group_concat(keyword) keywords 
from (
select * from tag_kwd_character_type
union all select * from tag_kwd_emotional_atmosphere
union all select * from tag_kwd_food_category
union all select * from tag_kwd_food_usage
union all select * from tag_kwd_is_retail_food
union all select * from tag_kwd_macro_occasion
union all select * from tag_kwd_micro_occasion
union all select * from tag_kwd_painpoint
union all select * from tag_kwd_text_topic
union all select * from tag_kwd_text_type
union all select * from tag_kwd_time
union all select * from tag_kwd_topic_relevance
union all select * from tag_kwd_when
union all select * from tag_kwd_where
) a 
group by 1,2
"""


In [6]:
df_dictionary = pd.read_sql(dict_sql, con = con)
df_dictionary['kw_ls'] = df_dictionary['keywords'].apply(lambda x: x.split(','))
print(df_dictionary.shape)
print(df_dictionary['keywords'].apply(lambda x: x.split(',').__len__()).sum())
df_dictionary.head()

(32453, 4)
203234


,tag_category,tag_value,keywords,kw_ls
0,主题相关性,1,"吃杨梅,仙居杨梅,仓鼠日常,杨梅,深圳汽车贴膜","[吃杨梅, 仙居杨梅, 仓鼠日常, 杨梅, 深圳汽车贴膜]"
1,主题相关性,2,"薯队长,消遣,牛肉粒,一大包,健身,健身日记,牛肉吃多了,City’super蛋卷,独立小包...","[薯队长, 消遣, 牛肉粒, 一大包, 健身, 健身日记, 牛肉吃多了, City’supe..."
2,主题相关性,3,"减脂期的福音,零食,感受,美美的环境,好吃,分享,鳝溪徒步,创意组合,小吃,和气桃桃,水煮菜...","[减脂期的福音, 零食, 感受, 美美的环境, 好吃, 分享, 鳝溪徒步, 创意组合, 小吃..."
3,人物类型,00后,00后,[00后]
4,人物类型,30岁及以上女性,"30岁的女人,40岁的女人","[30岁的女人, 40岁的女人]"


In [8]:
def _exact_matching(x, kw_ls, tag_name):
    tag:str = None 
    for kw in kw_ls:
        if len(kw) >1:
            if kw in x:
                tag = tag_name
                break
    return tag 
    
def tag_data_one_category(content, df_dictionary:pd.DataFrame, kw_ls_col = 'kw_ls', tag_value_col = 'tag_value'):
    tag_value_ls = []

    for tag_value, _dictionary in df_dictionary.groupby(tag_value_col):
        kw_ls = _dictionary[kw_ls_col].values[0]

        tag = _exact_matching(content, kw_ls, tag_value)

        if tag:
            tag_value_ls.append(tag)
    return tag_value_ls
        
def tag_data_multi_category(data, df_dictionary, categories, category_col = 'tag_category', content_col = 'content', kw_ls_col = 'kw_ls', tag_value_col = 'tag_value'):
    content = data[content_col]
    rst = []
    for cate in categories:
        _df_dictionary = df_dictionary[df_dictionary[category_col] == cate]
        tag = tag_data_one_category(content, _df_dictionary, kw_ls_col, tag_value_col)
        rst.append(','.join(tag))
    return rst

def tag_data(df_dictionary, data):
    data = data.copy()
    categories = df_dictionary['tag_category'].unique().tolist()
    data[categories] = data.progress_apply(tag_data_multi_category, axis = 1, result_type ='expand',df_dictionary = df_dictionary, categories= categories)
    return data

In [9]:
from tqdm import tqdm 
tqdm.pandas()

In [10]:
data_sql = """
select * , '202301' ds from preprocessed_q1_202301
union all select *, '202302' ds from preprocessed_q1_202302
union all select *, '202303' ds from preprocessed_q1_202303
union all select *, '202304' ds from preprocessed_q2_202304
union all select *, '202305' ds from preprocessed_q2_202305
union all select *, '202306' ds from preprocessed_q2_202306
union all select *, '202307' ds from preprocessed_q3_202307
union all select *, '202308' ds from preprocessed_q3_202308
union all select *, '202309' ds from preprocessed_q3_202309
union all select *, '202310' ds from preprocessed_q4_202310
union all select *, '202312' ds from preprocessed_q4_202311
union all select *, '202312' ds from preprocessed_q4_202312
"""

In [14]:
con = get_connection(database='xiaohongshu')
# n = pd.read_sql(f"select count(*) count from ({data_sql}) a", con = con)['count'].values[0]
chunksize = 100
step = int(n/chunksize)
n
for data in tqdm(pd.read_sql(data_sql, con = con, chunksize = chunksize), total = step):
    data = tag_data(df_dictionary, data)
    data[['id', 'ds', *df_dictionary['tag_category'].unique().tolist() ]].to_csv('data/tagged.csv', mode = 'a', index=False)

connecting to rm-uf6lfxk83h7onjmwvjo.mysql.rds.aliyuncs.com3306/xiaohongshu


KeyboardInterrupt: 

In [11]:
import multiprocessing as mp
from multiprocessing import Pool
con = get_connection(database='xiaohongshu')
n_process = mp.cpu_count()

def func(data):
    return tag_data(df_dictionary, data)
    
n = pd.read_sql(f"select count(*) count from ({data_sql}) a", con = con)['count'].values[0]
chunksize = 10
step = int(n/chunksize)
iterator = pd.read_sql(data_sql, con = con, chunksize = chunksize)

if __name__  == '__main__':
    pool = Pool(n_process)
    for d in tqdm(pool.imap_unordered(func, iterator), total = step):
        d[['id', 'ds', *df_dictionary['tag_category'].unique().tolist() ]].to_csv('data/tagged.csv', mode = 'a', index=False)

connecting to rm-uf6lfxk83h7onjmwvjo.mysql.rds.aliyuncs.com3306/xiaohongshu


  0%|          | 0/587164 [00:00<?, ?it/s]

In [28]:
data.loc[:1,'content'].values[0]

'先说万豪白金不烂大街哦[萌萌哒R]北上广江浙一带商旅人士多，三亚旅游的人多，可能大家看到的现象就觉得万豪白金烂大街，二三线城市还是非常非常好用的，具体权益和获取办法可以看我前面笔记[暗中观察R]\n\t\n[举手R]今天给大家讲一个小众且高级的酒店联盟：GHA\n大名鼎鼎的尼依格罗、三亚嘉佩乐就是这个联盟。\n划重点：几百块钱能升级的[二R]年有效期酒店🔝会籍，只有GHA钛金！（之前bug渠道已修复）\n\t\n[举手R]GHA钛金会员权益：\n1：客房连升两级，包括行政房或套房，房型差价1k+\n2：大部分酒店视情况供应免费双早和行政待遇，需要提前问酒店\n3：提前至11点入住，次日可延迟至16点退房。\n4：欢迎水果和点心\n5：免费游泳池健身房\n\t\n而且它秒升！秒升诶！[赞R]着急入住的快冲[赞R]'